In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sn
import mpld3
import pandas as pd
import imp
import numpy as np
import os
from sqlalchemy import create_engine
from shutil import copyfile
from openpyxl import load_workbook

sn.set_context('notebook')

# Recalculating OSPAR flow data

The discharge data previously submitted in Table 9 of the OSPAR template used the wrong format and needs to be re-entered (see e-mails from Eva received 15/08/2017 at 10.42 and 16/08/2017 at 10.44 for details). Eva has supplied a blank data entry template, which I've copied here:

C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet\Recalculate_OSPAR_Flows\Table 9 template.xlsx

For each year from 1990 to the present day, we need a version of this template based on the monitored and modelled flow datasets in the RID database.

This notebook develops code for filling-in these templates. With the exception of "Inner Oslofjord" and "Suldalslågen", which need to be considered separately (see later), the light-yellow rows in the template correspond to 10 of the "RID_11" monitoring stations. Data for these stations can be extracted directly. 

The bright-yellow cells, representing overall discharges to each of the four OSPAR regions, require a more sophisticated approach that combines modelled and monitored datasets. 

First establish a connection to the database.

In [2]:
# Connect to db
resa2_basic_path = (r'C:\Data\James_Work\Staff\Heleen_d_W\ICP_Waters\Upload_Template'
                    r'\useful_resa2_code.py')
resa2_basic = imp.load_source('useful_resa2_code', resa2_basic_path)
engine, conn = resa2_basic.connect_to_resa2()

# Import custom RID functions
rid_func_path = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
                 r'\Python\rid\notebooks\useful_rid_code.py')
rid = imp.load_source('useful_rid_code', rid_func_path)

## 1. Discharges from monitored locations

The code below extracts summary statistics for 10 of the RID_11 stations from 1990 to 2016 inclusive.

In [3]:
# Read site data for RID_11 and RID_36
in_xlsx = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
           r'\Data\RID_Sites_List.xlsx')
rid_11_df = pd.read_excel(in_xlsx, sheetname='RID_11')
rid_36_df = pd.read_excel(in_xlsx, sheetname='RID_36')

# Get just Suldalslågen from rid_36
rid_36_df = rid_36_df.query('station_id == 29781')

# Combine
mon_df = pd.concat([rid_11_df, rid_36_df], axis=0)

# Get OSPAR region for stations
sql = ("SELECT station_id, value "
       "FROM resa2.stations_par_values "
       "WHERE var_id = 262")
ospar_reg = pd.read_sql_query(sql, engine)
ospar_reg.columns = ['station_id', 'ospar_region']

# Join OSPAR regions to station data
mon_df = pd.merge(mon_df, ospar_reg, 
                  how='left', on='station_id')

# Get cols of interest
mon_df = mon_df[['station_id', 'station_code', 
                 'station_name', 'ospar_region']]

mon_df

,station_id,station_code,station_name,ospar_region
0,29615,VESENUM,Numedalslågen,SKAGERAK
1,29821,HOREVOS,Vosso(Bolstadelvi),NORTH SEA
2,29783,ROGEORR,Orreelva,NORTH SEA
3,29613,TELESKI,Skienselva,SKAGERAK
4,29614,VAGEOTR,Otra,SKAGERAK
5,29782,NOREVEF,Vefsna,NORWEGIAN SEA2
6,36225,OSLEALN,Alna,SKAGERAK
7,29617,ØSTEGLO,Glomma ved Sarpsfoss,SKAGERAK
8,29779,FINEALT,Altaelva,LOFOTEN-BARENTS SEA
9,29612,BUSEDRA,Drammenselva,SKAGERAK


In [4]:
# Container for results
df_list = []

# Loop over sites
for stn_id in mon_df['station_id']:
    # Get catch area for chem station
    sql = ("SELECT catchment_area FROM resa2.stations "
           "WHERE station_id = %s" % stn_id)
    area_df = pd.read_sql_query(sql, engine)    
    wc_area = area_df['catchment_area'].iloc[0]
    
    # Get linked discharge station
    sql = ("SELECT * FROM resa2.default_dis_stations "
           "WHERE station_id = %s" % stn_id)
    dis_df = pd.read_sql_query(sql, engine)
    dis_stn_id = dis_df['dis_station_id'].iloc[0]
    
    # Get catchment area for discharge station
    sql = ("SELECT area FROM resa2.discharge_stations "
           "WHERE dis_station_id = %s" % dis_stn_id)
    area_df = pd.read_sql_query(sql, engine)    
    dis_area = area_df['area'].iloc[0]

    # Get annual summary flow stats for this station
    sql = ("SELECT TO_CHAR(xdate, 'YYYY') as year, "
           "       AVG(xvalue) as mean, "
           "       MIN(xvalue) as min, " 
           "       MAX(xvalue) as max " 
           "FROM resa2.discharge_values "
           "WHERE dis_station_id = %s "
           "AND xdate >= date '1990-01-01' "
           "AND xdate <= date '2016-12-31' "
           "GROUP BY TO_CHAR(xdate, 'YYYY') "
           "ORDER BY year" % dis_stn_id)
    q_df = pd.read_sql_query(sql, engine) 
    
    # Set index
    q_df.index = q_df['year']
    del q_df['year']
    
    # Scale flows by area ratio
    q_df = q_df*wc_area/dis_area
    
    # Convert m3/s to 1000 m3/d
    q_df = q_df*60*60*24/1000
    
    # Reset index
    q_df.reset_index(inplace=True)
    
    # Add LTA and n_yrs
    q_df['lta'] = q_df['mean'].mean()
    q_df['n_yrs'] = len(q_df)
    
    # Add ospar region ID, n_sites and 'mean' cols
    q_df['area_id'] = stn_id
    q_df['stat'] = 'Mean'
    q_df['n_sites'] = 1
        
    # Re-order cols to match template
    q_df = q_df[['area_id', 'year', 'mean', 'lta', 'min',
                 'max', 'n_yrs', 'n_sites', 'stat']]
    
    # Add to results
    df_list.append(q_df)
    
# Combine to single df
q_mon_df = pd.concat(df_list, axis=0)    

q_mon_df.head()

,area_id,year,mean,lta,min,max,n_yrs,n_sites,stat
0,29615,1990,10119.152564,10311.415628,3412.955689,39299.530691,27,1,Mean
1,29615,1991,7884.893256,10311.415628,1352.807170,30882.317946,27,1,Mean
2,29615,1992,7838.387948,10311.415628,3106.161558,35247.383330,27,1,Mean
3,29615,1993,9867.524888,10311.415628,3256.978716,29220.191819,27,1,Mean
4,29615,1994,10823.288734,10311.415628,3106.161558,54601.346911,27,1,Mean


## 2. Modelled

The TEOTIL input file here:

K:\Avdeling\Vass\Teotil2\Norge_Regine\Regine.txt

incorporates data linking "regine" catchment IDs to NVE vassdrag numbers. In addition, the table `RESA2.RID_REGINE_DOWN` links each regine ID to an OSPAR region. These tables can be used to link the modelled NVE data to the OSPAR regions.

In [5]:
# Read cols of interest from Regine.txt
in_txt = r'K:\Avdeling\Vass\Teotil2\Norge_Regine\Regine.txt'
teo_df = pd.read_csv(in_txt, sep=';', comment='!', 
                     usecols=['Regine', 'VASSOMR'])
teo_df.columns = ['regine', 'vassdrag']

# Read data from RESA2.RID_REGINE_DOWN
sql = ("SELECT regine, area_fp "
       "FROM resa2.rid_regine_down")
ospar_reg = pd.read_sql_query(sql, engine) 

# Join
nve_df = pd.merge(ospar_reg, teo_df,
                  how='left', on='regine')

# Get cols of interest and simplify
nve_df = nve_df[['area_fp', 'vassdrag']]
nve_df.drop_duplicates(inplace=True)

## We already have monitored data for 10 of these vassdrags (see above)
## Remove these from consideration to avoid "double-counting"
## Get discharge station ids
#sql = ("SELECT * FROM resa2.default_dis_stations "
#       "WHERE station_id IN %s" % str(tuple(mon_df['station_id'].values)))
#dis_df = pd.read_sql_query(sql, engine)
#
## Get vassdrag numbers for montiored stations
#sql = ("SELECT nve_serienummer FROM resa2.discharge_stations "
#       "WHERE dis_station_id IN %s" % str(tuple(dis_df['dis_station_id'].values)))
#vass_nr = pd.read_sql_query(sql, engine)['nve_serienummer'].values
#vass_nr = [int(i.split('.')[0]) for i in vass_nr]
#
## Remove these vass_nrs from consideration in the modelled data
#nve_df = nve_df[~nve_df['vassdrag'].isin(vass_nr)]

nve_df.head()

,area_fp,vassdrag
0,24_90,68
9,24_90,69
88,24_90,70
220,24_90,71
240,24_90,72


In [6]:
def combine_flow_data(vass, reg):
    """ Function to combine time series for list of Vassdrags
        based on the NVE modelled data.
    
    Args:
        vass: List of strings. Vassdrags to combine 
        reg:  Str. ID for region
    
    Returns:
        Dataframe. Annual summary stats calculated from the 
        combined series (for 1990 to 2016)
    """
    # Get RESA2 station ID from vassdrag numbers
    sql = ("SELECT dis_station_id "
           "FROM resa2.discharge_stations "
           "WHERE nve_serienummer IN %s" % str(tuple(vass)))
    dis_ids = pd.read_sql_query(sql, engine) 
    
    assert len(dis_ids) == len(vass), 'Lengths of "vass" and "dis_ids" do not match.'
    
    # Sum flow data for all sites in OSPAR reg to create a single 
    # aggregated series    
    # Get annual summary flow stats for this region
    sql = ("SELECT TO_CHAR(xdate, 'YYYY') as year, "
           "       AVG(xvalue) as mean, "
           "       MIN(xvalue) as min, " 
           "       MAX(xvalue) as max " 
           "FROM ( "
           "  SELECT TRUNC(xdate) AS xdate, "
           "         SUM(xvalue) AS xvalue "
           "  FROM resa2.discharge_values "
           "  WHERE dis_station_id IN %s "
           "  AND xdate >= DATE '1990-01-01' "
           "  AND xdate <= DATE '2016-12-31' "
           "  GROUP BY TRUNC(xdate) "
           "  ORDER BY TRUNC(xdate)) "
           "WHERE xdate >= date '1990-01-01' "
           "AND xdate <= date '2016-12-31' "
           "GROUP BY TO_CHAR(xdate, 'YYYY') "
           "ORDER BY year" % str(tuple(dis_ids['dis_station_id'].values)))
    q_df = pd.read_sql_query(sql, engine) 

    # Set index
    q_df.index = q_df['year']
    del q_df['year']
        
    # Convert m3/s to 1000 m3/d
    q_df = q_df*60*60*24/1000
    
    # Reset index
    q_df.reset_index(inplace=True)
    
    # Add LTA and n_yrs
    q_df['lta'] = q_df['mean'].mean()
    q_df['n_yrs'] = len(q_df)
    
    # Add ospar region ID, n_sites and 'mean' cols
    q_df['area_id'] = reg
    q_df['stat'] = 'Mean'
    q_df['n_sites'] = len(vass)
    
    # Re-order cols to match template
    q_df = q_df[['area_id', 'year', 'mean', 'lta', 'min',
                 'max', 'n_yrs', 'n_sites', 'stat']]
    
    return q_df

In [7]:
# Container for results
df_list = []

# Loop over OSPAR areas
for osp_reg in nve_df['area_fp'].unique():
    # Get all vassdrags draining to this region
    vass = nve_df.query('area_fp == @osp_reg')['vassdrag'].values.astype(str)
    
    # Get stats
    q_df = combine_flow_data(vass, osp_reg)
    
    # Add to results
    df_list.append(q_df)
    
# Single calculation for the whole of norway
vass = nve_df['vassdrag'].values.astype(str)
q_df = combine_flow_data(vass, 'all_nor')
df_list.append(q_df)
    
# Combine to single df
q_mod_df = pd.concat(df_list, axis=0)    

q_mod_df.head()

,area_id,year,mean,lta,min,max,n_yrs,n_sites,stat
0,24_90,1990,403261.150362,303877.911598,120253.653043,1.055660e+06,27,67,Mean
1,24_90,1991,287394.261699,303877.911598,112613.966064,8.306305e+05,27,67,Mean
2,24_90,1992,367791.005856,303877.911598,110464.140269,1.108435e+06,27,67,Mean
3,24_90,1993,287751.669048,303877.911598,80757.900806,9.928839e+05,27,67,Mean
4,24_90,1994,313702.569079,303877.911598,88735.997318,6.832286e+05,27,67,Mean


## 3. Write results to template

The code below iterates over the output and writes the results to the Excel template. A new template is produced for each year.

In [8]:
# Years of interest
st_yr, end_yr = 1990, 2016

# Path to template
temp_path = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
             r'\Recalculate_OSPAR_Flows\Table 9 template.xlsx')

# Output folder
out_fold = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
            r'\Recalculate_OSPAR_Flows\Updated')

# Dict mappiong names in template to area IDs in dfs
name_dict = {'Orkla':29778,
             'Vefsna':29782,
             'Norwegian Sea (NO)':'91_170',
             'Alta':29779,
             'Barents Sea (NO)':'171_247',
             'Glomma':29617,
             'Drammenselva':29612,
             u'Numedalslågen':29615,
             'Skienselva':29613,
             'Otra':29614,
             'Inner Oslofjord':36225, # Assume just Alna for now(?)
             'Skagerrak (NO)':'1_23',
             'Orreelva':29783,
             u'Suldalslågen':29781,
             'Vosso':29821,
             'North Sea (NO)':'24_90',
             'Norway Total':'all_nor'}

# Loop over years
for year in range(st_yr, end_yr + 1):
    # Convert to string
    year = str(year)
    
    # Copy template
    out_path = os.path.join(out_fold, 'ospar_table_9_%s.xlsx' % year)
    copyfile(temp_path, out_path)
    
    # Open new file and get sheet
    wb = load_workbook(filename=out_path)
    ws = wb['9']
    
    # Set year
    ws['B2'] = int(year)
    
    # Loop over cells
    for item in ws['B12':'B28']:
        # Get cell properties
        cell = item[0]
        area = cell.value
        row = cell.row
        
        # Get area ID
        ar_id = name_dict[area]
        
        if ar_id != 999:
            # Get data from relevant df
            if isinstance(ar_id, int):
                # Monitored df
                df = q_mon_df.query('(area_id == @ar_id) and '
                                    '(year == @year)')
            else:
                # Modelled df
                df = q_mod_df.query('(area_id == @ar_id) and '
                                    '(year == @year)')
            
            assert len(df) == 1
            
            # Write values
            # 1. Mean
            ws.cell(column=5, row=row, value=df.iloc[0]['mean'])
            
            # 2. LTA
            ws.cell(column=7, row=row, value=df.iloc[0]['lta'])  
            
            # 3. Min
            ws.cell(column=9, row=row, value=df.iloc[0]['min']) 

            # 4. Max
            ws.cell(column=11, row=row, value=df.iloc[0]['max'])
            
            # 5. Years
            ws.cell(column=13, row=row, value=df.iloc[0]['n_yrs'])
            
            # 6. N_Sites
            ws.cell(column=15, row=row, value=df.iloc[0]['n_sites'])
            
            # 7. Stat
            ws.cell(column=17, row=row, value=df.iloc[0]['stat'])
    
    # Save
    wb.save(out_path)